In [1]:
import pandas as pd

data_file = 'data/synthetic_customer_data_train.csv'
out_file = 'data/synthetic_customer_data_train2.csv'
# data_file = 'data/synthetic_customer_data_test.csv'
# out_file = 'data/synthetic_customer_data_test2.csv'

zip_lookup_file = "data/cities.csv"
state_lookup_file = "data/states.csv"

data = pd.read_csv(data_file)
data['ZIP']=data.ADDRESS.apply(lambda x: x[-5:])


In [2]:
# Load zip code lookup table
# us_zip = pd.read_csv('US_ZIP.csv', dtype={"ZIP":"string"})

# ignore the "CITY" column
cities = pd.read_csv(zip_lookup_file, dtype={'ZIP':'string'})[["ZIP","STATE","LAT","LNG"]]

# merge lookup
data = data.merge(cities, on='ZIP', how='left')

In [3]:
# correct the State code in Address column
# print("{} {}".format(data.STATE, data.ZIP))
# data["ADDRESS2"] = None

# update the ADDRESS column with corrected STATE ZIP info
# data["ADDRESS2"] = data.apply(lambda r: "{}{} {}".format(r["ADDRESS"][0:-8], r["STATE"], r["ZIP"]) if pd.isna(r["LAT"]) == False else None, axis=1)
data["ADDRESS"] = data.apply(lambda r: "{}{} {}".format(r["ADDRESS"][0:-8], r["STATE"], r["ZIP"]) if pd.isna(r["LAT"]) == False else r["ADDRESS"], axis=1)

In [4]:
# use states as the update table (lookup)
states = pd.read_csv(state_lookup_file)[["STATE","LAT","LNG"]]
states.set_index('STATE', inplace=True)

# update the "STATE2" column for all rows without latitude/longitude and set index
data.loc[data.LAT.isna(),"STATE2"] = data.ADDRESS.apply(lambda x: x[-8:-6])

# update NA values to 0 (not necessary)
# data.loc[data.LAT.isna(), ["LAT","LNG"]] = [0, 0]

# set index on STATE2 to join with "states" dataframe
data.set_index('STATE2', inplace=True)

In [5]:
# update just the blank lat/lng rows
data.update(states)

# reset index
data.reset_index(inplace=True)

# remove 

In [6]:
# delete working columns
del data["STATE2"]
del data["STATE"]
del data["ZIP"]

In [7]:
print("{} rows left with no Lat/Lng values".format(len(data[data.LAT.isna()] )))

775 rows left with no Lat/Lng values


In [8]:
data.to_csv(out_file)